# Monoprotic acid

Links:  
[Wikipedia](https://en.wikipedia.org/wiki/Acid_dissociation_constant)  
[Chem buddy](https://www.chembuddy.com/?left=pH-calculation&right=toc)  
[Dissociation constants 1](https://chem.libretexts.org/Bookshelves/Ancillary_Materials/Reference/Reference_Tables/Equilibrium_Constants/E1%3A_Acid_Dissociation_Constants_at_25C)  
[aqion pH masurements](https://www.aqion.de/site/191)  
[aqion pH calculator](http://www.aqion.onl/)

Equations:
HA -> A- + H+

Equilibrium constant  
$K_a = \frac{[A^-][H^+]}{[HA]}$

Mass balance:  
$C_a = [HA] + [A^-]$

Charge balance (we have neutral solution):  
$[OH^-] + [A^-] = [H^+]$

Full solution to above system:
$[H^+]^3-K_a [H^+]^2 - (C_aK_a+K_w)[H^+]-K_aK_w=0$

Strong acid approximation:  
$[A^-] \approx C_a$  
$[H^+] = \frac{C_a}{2}(1\pm\sqrt{1-\frac{4K_w}{C_a^2}})$  
Even stronger (less accurate) approximation:  
$[H^+] \approx C_a$


### Data:  
Water dissociation constant:  
$K_w=10^{-14}$ (at 25 C)

In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

## Solvers

In [12]:
def bisect(f, a, b, eps=1e-6, maxiter=15, m = 1, verbose = False):
    if f(a)*f(b) >= 0:
        print("Error func")
        return
    i = 0
    while (abs(a-b) > eps and (i < maxiter)):
        zero = (b + a) / 2
        if f(zero)*f(b) < 0:
            a = zero
        else:
            b = zero
        i += 1
    return zero
    
def newton_iteration(f, fder, x0, eps=1e-15, maxiter=1000, m = 1, verbose = False):
    """Find a root of $f(x) = 0$ via Newton's iteration starting from x0.
    I've added the modified version.
    
    Parameters
    ----------
    f : callable
        The function to find a root of.
    fder : callable
        The derivative of `f`.
    x0 : float
        Initial value for the Newton's iteration.
    eps : float
        The target accuracy. 
        The iteration stops when the distance between successive iterates is below `eps`.
        Default is 1e-5.
    maxiter : int
        The maximum number of iterations (default is 1000.)
        Iterations terminate if the number of iterations exceeds `maxiter`.
        This parameter is only needed to avoid infinite loops if iterations wander off.
        
    Returns
    -------
    x : float
        The estimate for the root.
    niter : int
        The number of iterations.
    """
    x_prev = x0+2*eps
    x_new = x0
    i=0
    while(abs(x_prev - x_new) > eps and i < maxiter):
        x_prev = x_new
        x_new = x_prev - m * (f(x_prev)/fder(x_prev))
        #x_prev = x
        if i < 100 and verbose:
            print("i: ",i, "old: ", x_prev, "new: ", x_new)
        i += 1
        
    return x_new, i


In [13]:
# initial data
# acid molar concentration
HCl_data = {
    "c_a" : 0.01, # mol/L
    "K_a" : [7.9e5]
}
Acetic_data = {
    "c_a" : 0.01,
    "K_a" : [1.75e-5],
    "pK_a" : [4.756]
} 
hydrocyanic_data = {
    "c_a" : 0.1,
    "pK_a" : [9.31]
}

In [14]:
def calc_pH_acid(c_a, K_a, K_w = 10e-14, verbose = True, plot= False):
    
    solutions = []
    
    # strong approximations
    strong_approx = c_a/2*(1+np.sqrt(1-4*K_w/c_a**2))
    strong_strong_approx = c_a
    
    # weak approximations
    weak_approx = (-K_a + np.sqrt(K_a**2 + 4 * K_a * c_a)) / 2
    weak_weak_approx = K_a / 2 + np.sqrt(K_a * c_a)
    weak_weak_weak_approx = np.sqrt(K_a * c_a)
    
    # defining equation and derivative
    def equation(x):
        return x**3 + K_a * x**2 - (c_a * K_a) * x - K_a * K_w
    def equation_der(x):
        return 3*x**2 + 2 * K_a * x - (c_a * K_a)
    
    #plotting data
    if plot:
        x = np.linspace(-0.001, 0.0561, 100)
        y = x**3+K_a*x**2-(c_a*K_a)*x-K_a*K_w
        fig, ax = plt.subplots()
        ax.plot(x, y)
        ax.axhline(0, color = "k")
        
    exact = bisect(equation, 1e-10, 0.2)
    better = newton_iteration(equation, equation_der, exact)
    solutions.append(better[0])
    solutions.append(strong_approx)
    solutions.append(strong_strong_approx)
    solutions.append(weak_approx)
    solutions.append(weak_weak_approx)
    solutions.append(weak_weak_weak_approx)
    solutions = -np.log10(np.array(solutions))
    
    if verbose:
        txt = "{:7.4f} {:7.4f} {:7.4f} {:7.4f} {:7.4f} {:7.4f}"
        print(txt.format(*solutions))
    return -np.log10(exact), -np.log10(better[0])
c_a = HCl_data["c_a"]
K_a = HCl_data["K_a"][0]
_ = (calc_pH_acid(c_a, K_a))
c_a = Acetic_data["c_a"]
K_a = 10**(-Acetic_data["pK_a"][0])
_ = (calc_pH_acid(c_a, K_a, plot=False))
c_a = hydrocyanic_data["c_a"]
K_a = 10**(-hydrocyanic_data["pK_a"][0])
_ = (calc_pH_acid(c_a, K_a))

 2.0000  2.0000  2.0000  2.0000 -5.5967 -1.9488
 3.3871  2.0000  2.0000  3.3871  3.3690  3.3780
 5.1550  1.0000  1.0000  5.1550  5.1550  5.1550


In [65]:
# molarities are in mol/L
# Input data
molarities_dict = {
    "Acetic acid" : 0.01,
    #"Citric acid" : 0.1,
    #"Phosphoric acid" : 0.003,
}

# REquired data.
pKa_dict = {
    "Acetic acid" : np.array([4.756]),
    "Citric acid" : np.array([3.128, 4.761, 6.396]),
    "Glycine" : np.array([2.35]),
    "Hydrocloric acid" : np.array([-4]),
    "Hydorcyanic acid" : np.array([9.31]),
    "Nitric acid" : np.array([-1]),
    "Phosphoric acid" : np.array([2.147, 7.207, 12.346]),
    "Succinic acid" : np.array([4.2, 5.6]),
    "TRIS" : np.array([])
}

# charges of conjugate (bases acid -> conjuget_base + H^+ ---- HA -> A^- + H^+)
charges_dict = {
    "OH" : np.array([-1]),
    "H" : np.array([1]),
    "Acetic acid" : np.array([-1]),
    "Citric acid" : np.array([-1, -2, -3]),
    "Glycine" : np.array([-1]),
    "Hydrocloric acid" : np.array([-1]),
    "Hydorcyanic acid" : np.array([-1]),
    "Nitric acid" : np.array([-1]),
    "Phosphoric acid" : np.array([-1, -2, -2]),
    "Succinic acid" : np.array([-1, -2]),
}

def ion_strength(ion_molarities_dict, charge_dict, c_Na = 0.0, c_Cl = 0.0):
    ion_strength = 0
    for species in ion_molarities_dict:
        ion_strength += (charge_dict[species]**2 * ion_molarities_dict[species]).sum()
        
    return (ion_strength + c_Na + c_Cl) / 2.0

def electro_neutrality_error(ion_molarities_dict, charge_dict, c_Na = 0.0, c_Cl = 0.0):
    error = 0
    for species in ion_molarities_dict:
        #print("Electro neu calc: {} {} {}".format(species, charge_dict[species], ion_molarities_dict[species]))
        error += (charge_dict[species] * ion_molarities_dict[species]).sum()
        
    return error + c_Na - c_Cl

def ion_strength_error(ion_strength_guess, ion_molarities_dict, charge_dict, c_Na = 0.0, c_Cl = 0.0):
    return ion_strength_guess - ion_strength(ion_molarities_dict, charge_dict, c_Na = 0.0, c_Cl = 0.0)

def calculate_molarities(molarities_dict, pKa_dict, charge_dict, pH, ion_strength, correct_for_IS = True):
    """Calculates molarites of all charged species (ions) in water solution at given pH and ion_stregth."""
    c_H = 10**(-pH)
    K_w = 10**(-14)
    c_OH = K_w / c_H
    ion_molarities = {
        "H" : np.array([c_H]),
        "OH" : np.array([c_OH]),
    }
    for species in molarities_dict:
        ion_molarities[species] = np.zeros(len(pKa_dict[species]))
        denom = 1
        term = 1
        for charge, pK_a in zip(charge_dict[species], pKa_dict[species]):
            pK_a_eff = pK_a
            if correct_for_IS:
                pK_a_eff += 2 * (charge) * (0.5114 * np.sqrt(ion_strength) / (1 + np.sqrt(ion_strength)) - 0.1 * ion_strength)
            term *= 10**(-pK_a_eff) / c_H
            denom += term
        molarity = molarities_dict[species] / denom
        for i, pK_a in enumerate(pKa_dict[species]):
            pK_a_eff = pK_a
            if correct_for_IS:
                pK_a_eff += 2 * (charge) * (0.5114 * np.sqrt(ion_strength) / (1 + np.sqrt(ion_strength)) - 0.1 * ion_strength)
            molarity *= 10**(-pK_a_eff) / c_H
            ion_molarities[species][i] = molarity
    return ion_molarities

c_Na = 0.30
c_Cl = 0.0

print(ion_strength(molarities_dict, charges_dict, c_Na, c_Cl))
print(electro_neutrality_error(molarities_dict, charges_dict, c_Na, c_Cl))

print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 3, 2, correct_for_IS=False))
print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 3, 2))
print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 5, 2, correct_for_IS=False))
print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 5, 2))
print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 6, 2, correct_for_IS=False))
print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 6, 2))
print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 7, 2, correct_for_IS=False))
print(calculate_molarities(molarities_dict, pKa_dict, charges_dict, 7, 2))

0.155
0.29
{'H': array([0.001]), 'OH': array([1.e-11]), 'Acetic acid': array([0.00017236])}
{'H': array([0.001]), 'OH': array([1.e-11]), 'Acetic acid': array([0.00026993])}
{'H': array([1.e-05]), 'OH': array([1.e-09]), 'Acetic acid': array([0.00636876])}
{'H': array([1.e-05]), 'OH': array([1.e-09]), 'Acetic acid': array([0.00735045])}
{'H': array([1.e-06]), 'OH': array([1.e-08]), 'Acetic acid': array([0.00946059])}
{'H': array([1.e-06]), 'OH': array([1.e-08]), 'Acetic acid': array([0.00965208])}
{'H': array([1.e-07]), 'OH': array([1.e-07]), 'Acetic acid': array([0.00994331])}
{'H': array([1.e-07]), 'OH': array([1.e-07]), 'Acetic acid': array([0.00996408])}


In [61]:
print("##### Benchmark pH")
c_a = Acetic_data["c_a"]
print("C_a: {} M".format(c_a))
K_a = 10**(-Acetic_data["pK_a"][0])
_ = (calc_pH_acid(c_a, K_a, plot=False))
print("end\n")

c_Na = 0.0
c_Cl = 0.0
print(molarities_dict, "\n")

ion_molarities_dict = calculate_molarities(molarities_dict, pKa_dict, charges_dict, 2.62, 0.0572333372, correct_for_IS=True)
print(ion_molarities_dict)
print(electro_neutrality_error(ion_molarities_dict, charges_dict, c_Na, c_Cl))
print(ion_strength(ion_molarities_dict, charges_dict), "\n")

ion_molarities_dict = calculate_molarities(molarities_dict, pKa_dict, charges_dict, 4, 2, correct_for_IS=False)
print(ion_molarities_dict)
print(electro_neutrality_error(ion_molarities_dict, charges_dict, c_Na, c_Cl))

ion_molarities_dict = calculate_molarities(molarities_dict, pKa_dict, charges_dict, 3.38, 2, correct_for_IS=False)
print(ion_molarities_dict)
print(electro_neutrality_error(ion_molarities_dict, charges_dict, c_Na, c_Cl),"\n")

ion_molarities_dict = calculate_molarities(molarities_dict, pKa_dict, charges_dict, 3.3871, 2, correct_for_IS=False)
print(ion_molarities_dict)
print(electro_neutrality_error(ion_molarities_dict, charges_dict, c_Na, c_Cl))

##### Benchmark pH
C_a: 0.01 M
 3.3871  2.0000  2.0000  3.3871  3.3690  3.3780
end

{'Acetic acid': 0.01} 

calc ion molarities
{'H': array([0.00239883]), 'OH': array([4.16869383e-12]), 'Acetic acid': array([0.00011096])}
0.00228787466952717
0.001254895584255905 

calc ion molarities
{'H': array([0.0001]), 'OH': array([1.e-10]), 'Acetic acid': array([0.00149217])}
-0.0013921715590912663
calc ion molarities
{'H': array([0.00041687]), 'OH': array([2.39883292e-11]), 'Acetic acid': array([0.00040374])}
1.312915653065976e-05 

calc ion molarities
{'H': array([0.00041011]), 'OH': array([2.43837221e-11]), 'Acetic acid': array([0.00041012])}
-1.2377668000415013e-08


In [66]:
def bisect(f, a, b, eps=1e-6, maxiter=15, m = 1, verbose = False):
    if f(a)*f(b) >= 0:
        print("Error func")
        return
    i = 0
    while (abs(a-b) > eps and (i < maxiter)):
        zero = (b + a) / 2
        if f(zero)*f(b) < 0:
            a = zero
        else:
            b = zero
        i += 1
    return zero

def calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, ion_strength, c_Na, c_Cl, correct_for_IS=False):

    eps = 1e-8
    i = 0
    error = 1
    pH_low = pH_low
    pH_high = pH_high
    while(abs(error) > eps and i < 100):
        pH = (pH_low + pH_high) / 2
        ion_molarities_dict = calculate_molarities(molarities_dict, pKa_dict, charges_dict, pH, ion_strength, correct_for_IS=correct_for_IS)
        error = electro_neutrality_error(ion_molarities_dict, charges_dict, c_Na, c_Cl)
        if error > 0:
            pH_low = pH
        else:
            pH_high = pH
        #print("i: {} pH {:.4f} error: {:.4f}".format(i, pH, error))
        i += 1
    return pH
    
def calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, IS_low, IS_high, pH, c_Na, c_Cl, correct_for_IS = True):
    eps = 1e-8
    i = 0
    error = 1
    while(abs(error) > eps and i < 100):
        IS = (IS_low + IS_high) / 2
        ion_molarities_dict = calculate_molarities(molarities_dict, pKa_dict, charges_dict, pH, IS, correct_for_IS=correct_for_IS)
        error = IS - ion_strength(ion_molarities_dict, charges_dict, c_Na, c_Cl)
        if error < 0:
            IS_low = IS 
        else:
            IS_high = IS
        #print("i: {} IS {:.4f} error: {:.4f}".format(i, IS, error))
        i += 1
    return IS

def calculate_pH_IS(molarities_dict, c_Na, c_Cl, initial_pH, pKa_dict, charges_dict, verbose=True):
    IS_min = 0.00001
    IS_max = 5.5
    pH_min = 0.0
    pH_max = 14.0
    error = 1e-5
    
    # TODO some smart guess of initial_pH
    IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, IS_min, IS_max, initial_pH, c_Na, c_Cl, True)
    pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_min, pH_max, IS, c_Na, c_Cl, True)
    if verbose:
        print(f"Initial values, pH: {pH:8.5f} IS: {IS:8.5f}")
    
    iteration = 1
    previous_pH = initial_pH
    while(abs(pH - previous_pH) > error and iteration < 1000):
        IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, IS_min, IS_max, pH, c_Na, c_Cl, True)
        previous_pH = pH
        pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_min, pH_max, IS, c_Na, c_Cl, True)
        if verbose:
            print(f"step: {iteration} pH: {pH:8.5f} IS: {IS:8.5f}")
            print(f"error: {abs(pH-previous_pH):9.7f}")
        iteration += 1
    return pH, IS

  

In [67]:
c_Na = 0.0
c_Cl = 0.0

# molarities are in mol/L
molarities_dict = {
    "Acetic acid" : 0.1,
    "Citric acid" : 0.1,
    "Phosphoric acid" : 0.3,
}
#%timeit pH, IS = calculate_pH_IS(molarities_dict, c_Na, c_Cl, 1, pKa_dict, charges_dict, verbose =False)
pH, IS = calculate_pH_IS(molarities_dict, c_Na, c_Cl, 1, pKa_dict, charges_dict, verbose =False)
print("result: ", pH, IS)

result:  1.1537572667002678 0.07018900328360497


In [56]:
c_Na = 1.2
c_Cl = 1.2

# molarities are in mol/L
molarities_dict = {
    "Acetic acid" : 0.1,
    "Citric acid" : 0.1,
    "Phosphoric acid" : 0.3,
}

pH_low = 0
pH_high = 14
print(molarities_dict)
for IS in [1, 0.1, 0.01, 0.001, 0.0001, 0.000010]:
    calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, IS, c_Na, c_Cl, True)

pH = 5
IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, 0.00001, 5.5, pH, c_Na, c_Cl, True)
pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, IS, c_Na, c_Cl, True)
print("pH: {:8.5f} IS: {:8.5f}".format(pH, IS))
IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, 0.00001, 5.5, pH, c_Na, c_Cl, True)
pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, IS, c_Na, c_Cl, True)
print("pH: {:8.5f} IS: {:8.5f}".format(pH, IS))
IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, 0.00001, 5.5, pH, c_Na, c_Cl, True)
pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, IS, c_Na, c_Cl, True)
print("pH: {:8.5f} IS: {:8.5f}".format(pH, IS))
IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, 0.00001, 5.5, pH, c_Na, c_Cl, True)
pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, IS, c_Na, c_Cl, True)
print("pH: {:8.5f} IS: {:8.5f}".format(pH, IS))
IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, 0.00001, 5.5, pH, c_Na, c_Cl, True)
pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, IS, c_Na, c_Cl, True)
print("pH: {:8.5f} IS: {:8.5f}".format(pH, IS))
IS = calculate_ion_strength_bisect(molarities_dict, pKa_dict, charges_dict, 0.00001, 5.5, pH, c_Na, c_Cl, True)
pH = calculate_pH_bisect(molarities_dict, pKa_dict, charges_dict, pH_low, pH_high, IS, c_Na, c_Cl, True)
print("pH: {:8.5f} IS: {:8.5f}".format(pH, IS))

{'Acetic acid': 0.1, 'Citric acid': 0.1, 'Phosphoric acid': 0.3}
calc ion molarities
4.4446
2.8166
4.1382
5.4618
1.8356
6.5842
11.7232
{'H': array([1.e-07]), 'OH': array([1.e-07]), 'Acetic acid': array([0.09972242]), 'Citric acid': array([1.62324720e-05, 2.41871481e-02, 8.35188803e-01]), 'Phosphoric acid': array([1.70456987e-01, 4.44032551e-01, 8.39875299e-06])}
i: 0 pH 7.0000 error: -3.7122
calc ion molarities
4.4446
2.8166
4.1382
5.4618
1.8356
6.5842
11.7232
{'H': array([0.00031623]), 'OH': array([3.16227766e-11]), 'Acetic acid': array([0.01020161]), 'Citric acid': array([0.29139535, 0.13730382, 0.00149928]), 'Phosphoric acid': array([6.00988379e-01, 4.95069390e-04, 2.96118864e-12])}
i: 1 pH 3.5000 error: -1.1824
calc ion molarities
4.4446
2.8166
4.1382
5.4618
1.8356
6.5842
11.7232
{'H': array([0.01778279]), 'OH': array([5.62341325e-13]), 'Acetic acid': array([0.00020162]), 'Citric acid': array([3.31372677e-02, 2.77662159e-04, 5.39158797e-08]), 'Phosphoric acid': array([2.77065730e-0

In [20]:
3.387

3.387